# **과제2 : 챗봇 만들기2**

## 0.미션

* 예비 에이블러들을 위한 QA 챗봇 모델 만들기2
    * Vector DB에 데이터 추가하기
    * Retriever, memory, LLM를 연결하기
    * 실행시 이력 DB 생성하고 기록하기
    * test

## **1.환경준비**

### (1) 라이브러리 Import

In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
from datetime import datetime

import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [ ]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key) 

* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [3]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키' 
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Vector DB 만들기**

* 데이터 로딩 
    * 1일차에서 제공한 csv 파일의 구조를 그대로 이용
    * 에이블스쿨 홈페이지 FAQ 데이터 수집(https://aivle.kt.co.kr/home/brd/faq/main?mcd=MC00000056)
        * 모든 질문을 csv 형태로 저장
    * 데이터프레임으로 저장하기

* 벡터 데이터베이스
    * 1일차 벡터 데이터베이스를 그대로 이용
        * Embedding 모델 : text-embedding-ada-002
        * DB 경로 : ./db



In [40]:
embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")
database = Chroma(persist_directory = "./db", # 경로 지정(현 위치에서 db 폴더 생성)
                  embedding_function = embeddings) # 임베딩 벡터로 만들 모델 지정

* 데이터 입력
    * 기존 입력을 모두 제거하고 추가 사항만 모두 입력 → DB의 기존값에 추가
    * meta data로 '구분' 칼럼 값 추가하기

In [39]:
# # DB 내용 다 지우기
# ids = database.get()
# database.delete(ids = ids['ids'])

In [42]:
len(database)

10

In [43]:
data = pd.read_csv('aivleschool_qa2.csv', encoding='utf-8')
# 한문, 한글, 일본어 등 한 글자 표현하는 데 2byte 드는 문자들: UTF-8로 읽어와야 함
data.head()
# 구분은 메타데이터에

,구분,QA
0,모집/선발,K-DT가 아닌 국비과정 참여 중인데 지원할 수 있나요?\n기존에 수강하신 과정이 ...
1,모집/선발,이전에 불합격했을 경우 이번에 다시 지원할수 있나요?\n지원자격에 해당되는 경우 다...
2,모집/선발,합격 후 과정을 포기하면 재지원이 불가능한가요?\n교육 등록신청을 하지 않거나 교육...
3,모집/선발,추가합격제도가 있나요?\n현재 KT 에이블스쿨은 추가합격 제도를 운영하고 있지 않습...
4,모집/선발,면접은 어떻게 진행되나요?\nKT 에이블스쿨은 별도의 면접 과정이 없습니다


In [44]:
print(len(data))

58


In [45]:
text_list = data['QA'].tolist()
meta_list = data['구분'].tolist()

# meta_list의 각 항목을 딕셔너리로 변환합니다.
doc = [Document(page_content=text_list[i], metadata={'구분': meta_list[i]}) for i in range(len(data))]

# 데이터베이스에 문서를 추가합니다.
ind = database.add_documents(doc)

* 입력된 데이터 조회

In [46]:
database.get(ind)

{'ids': ['022b935d-f7c6-4d79-ad72-88d20e57f7ce',
  '031e5c6f-62c0-4d50-b852-fa656fcd490e',
  '050bf9d3-2afd-4bcc-8e36-46aecc3e526e',
  '0c340707-3c1e-436b-b837-080037242487',
  '0fecfec9-527c-414e-b919-3c017f678b93',
  '1738d071-cfce-420f-810b-ade88f059dae',
  '19e2178b-c8ca-45df-9315-dfcc0afe6170',
  '2130001f-8684-4f7c-94df-dfa4fd5a4cce',
  '21998f4b-e126-4d8b-8c70-c319e1c2530d',
  '220d4fd1-54e1-4ace-a792-36b69157007a',
  '22a95bc8-1bfe-466f-a2fc-350a4ade90bf',
  '24960876-5add-49c4-a17e-e634c98561a5',
  '378c16f4-5a41-4645-b8c4-51d293abb011',
  '3b451aa8-964a-4f38-9702-fec618180cc4',
  '3fc1a03e-e61b-4688-8846-58f3f19e6f74',
  '404ff007-c9bb-4611-9a4d-7fb5ed2e39ad',
  '4223a42a-3a62-4719-837e-d62b1e4cdab6',
  '46a92028-ac83-413d-a316-0515edce7db5',
  '4ddfa3b1-f2c4-430c-a875-6a51d817b726',
  '53020c14-8be8-4b15-b327-4f85598f9208',
  '53a602e5-a3d4-4307-9360-88f0eee043f4',
  '54e210d9-82a0-4427-af17-613360f47fff',
  '5609bb0d-1f92-4d0d-b8b3-3a799137546d',
  '56b20458-1db0-4759-8ba8-

In [47]:
len(database)

68

## **3.RAG+LLM모델**

* 모델 : ConversationalRetrievalChain
    * LLM 모델 : gpt-3.5-turbo
    * retriever : 벡터DB
        * 유사도 높은 문서 3개 가져오도록 설정
    * memory 사용
* 요구사항
    * 질문 history 관리를 위한 이력 저장 DB 생성
        * DB 명 : db_chatlog
        * 테이블 명 : history
            * id INTEGER PRIMARY KEY : 이렇게 설정하면 자동증가 값으로 채워짐
            * datetime TEXT : 질문시점 yyyy-mm-dd hh:mi:ss
            * query TEXT : 질문
            * sim1 REAL : 첫번째 문서의 유사도 점수
            * sim2 REAL : 두번째 문서의 유사도 점수
            * sim3 REAL : 세번째 문서의 유사도 점수
            * answer TEXT : 답변
        * 유사도 점수는 similarity_search_with_score 메서드를 이용해서 저장해야 함
        * 질문과 답변이 진행될 때마다 history 테이블에 데이터 입력

* 관리용 DB, 테이블 생성

In [53]:
path = './db/db_chatlog.db'
conn = sqlite3.connect(path)

# 커서 객체 생성
cursor = conn.cursor()

# test 테이블 생성
cursor.execute('''
CREATE TABLE IF NOT EXISTS log (
    id INTEGER PRIMARY KEY,
    datetime TEXT NOT NULL,
    query TEXT NOT NULL,
    sim1 REAL NOT NULL,
    sim2 REAL NOT NULL,
    sim3 REAL NOT NULL,
    answer TEXT NOT NULL
)
''') # REAL = FLOAT(24)

# 변경사항 커밋 (저장)
conn.commit()

# 연결 종료
conn.close()

* 모델 선언

In [54]:
chat = ChatOpenAI(model="gpt-3.5-turbo",
                  temperature = 1.0,
                  max_tokens = 200)

k = 3
retriever = database.as_retriever(search_kwargs={"k": k})

# 대화 메모리 생성
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer",
                                  return_messages=True) # 대화 시작할 때 실행돼야 함

# ConversationalRetrievalQA 체인 생성
qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory,
                                           return_source_documents=True, output_key="answer")

* 모델 사용 및 이력 확인

In [61]:
from datetime import datetime

# 현재 시간
dt = datetime.now()
dt = dt.strftime('%Y-%m-%d %H:%M:%S') # 날짜/시간 → 문자열

query = input()
query = query.strip()
result = qa(query)
answer = result["answer"]

similarities = database.similarity_search_with_score(query, k = k) # 데이터베이스에서 유사도가 높은 문서를 가져옴

sim1 = round(similarities[0][1],5)
sim2 = round(similarities[1][1],5)
sim3 = round(similarities[2][1],5)

print(dt, query, sim1, sim2, sim3, answer, sep='\n')

# ① 연결
conn = sqlite3.connect(path)

# ② 작업 : to_sql
data = pd.DataFrame({'datetime': [dt], 'query': query, 'sim1': sim1, 'sim2': sim2, 'sim3': sim3, 'answer': [answer]})
data.to_sql('history', conn, if_exists='append', index=False) # history 테이블이 있으면 insert, 없으면 생성

2024-06-04 20:40:39
잡페어는 뭐야?
0.37538
0.43599
0.44673
잡페어는 취업 박람회로, 기업들이 직원을 채용하려고 하는 곳에서 구직자들이 기업들과 만나 서로 소통하고 취업 기회를 찾을 수 있는 행사입니다.


1

In [62]:
# 확인
df = pd.read_sql('SELECT * FROM history', conn)
display(df)

# ③ 연결 종료
conn.close()

,datetime,query,sim1,sim2,sim3,answer
0,2024-06-04 20:39:55,안녕,0.40306,0.41778,0.42539,안녕하세요! 궁금하신 점 있으신가요?
1,2024-06-04 20:40:01,에이블스쿨이 뭐야?,0.30322,0.34889,0.34914,"에이블스쿨은 KT와 정부가 함께 하는 프로그램으로, 미래인재육성과 청년 IT/SW/..."
2,2024-06-04 20:40:17,취업 기회를 위한 다양한 프로그램으로는 뭐가 있어?,0.27213,0.27460,0.31159,KT 에이블스쿨에서 제공하는 취업 기회를 위한 다양한 프로그램으로는 Job Sear...
3,2024-06-04 20:40:39,잡페어는 뭐야?,0.37538,0.43599,0.44673,"잡페어는 취업 박람회로, 기업들이 직원을 채용하려고 하는 곳에서 구직자들이 기업들과..."
